<a href="https://colab.research.google.com/github/ChiaoYunTing/ADA-Group15/blob/main/neural_loan_data_Ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:

from google.colab import files

uploaded = files.upload()

Saving loan_data.xlsx to loan_data.xlsx


In [8]:
# upload an in-built Python (OK semi-in-built) dataset
import pandas as pd
import numpy as np

df = pd.read_excel('loan_data.xlsx')

In [9]:
target = df.pop('loan_is_bad')
features = df
tf_dataset = tf.data.Dataset.from_tensor_slices((features.values, target.values))

In [10]:
n = len(list(tf_dataset))
train_size = int(n * 0.8)
train_dataset = tf_dataset.take(train_size)
test_dataset = tf_dataset.skip(train_size)

In [12]:
train_batch = train_dataset.batch(50) # batch size = 50
features, labels = next(iter(train_batch)) # iterate through each batch at training time

# print the first 20 records (rows) of features and labels
print(features[0:20])
print(labels[0:20])

tf.Tensor(
[[1.12000000e+04 3.60000000e+01 6.62000000e+00 3.43890000e+02
  1.00000000e+00 4.00000000e+00 1.08000000e+05 1.00000000e+00
  2.00000000e+00 2.00000000e+00 1.25200000e+01 0.00000000e+00
  0.00000000e+00 9.00000000e+00 0.00000000e+00 3.78220000e+04
  2.10000000e+01 1.16760600e+04 1.05045300e+04 1.17153000e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.43890000e+02
  0.00000000e+00 0.00000000e+00 1.24047306e+02 9.69776610e+01
  9.69776610e+01]
 [1.00000000e+04 3.60000000e+01 1.11400000e+01 3.28060000e+02
  2.00000000e+00 1.00000000e+00 6.50000000e+04 1.00000000e+00
  1.00000000e+00 2.00000000e+00 9.58000000e+00 0.00000000e+00
  0.00000000e+00 9.00000000e+00 0.00000000e+00 1.66230000e+04
  1.10000000e+01 4.62034000e+03 2.71084000e+03 8.97600000e+02
  0.00000000e+00 1.01190000e+03 1.01190000e+01 3.28060000e+02
  0.00000000e+00 0.00000000e+00 1.55059133e+02 1.19940867e+02
  1.19940867e+02]
 [8.00000000e+03 3.60000000e+01 1.62900000e+01 2.82410000e+02
  3.00000000e+00 5.0000

In [13]:

feature_count = features.shape[1]#determine number of features
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(feature_count,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])#dropout rate could be 0.2-0.5 and it is for reducing overfitting

In [14]:
# compile the model
model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001,  weight_decay=0.005),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])#learning_rate and weight_decay are for reducing overfitting(idk whether it is overlapped with dropout rate)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1920      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4033 (15.75 KB)
Trainable params: 4033 (15.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
history = model.fit(features, labels, epochs=100)

Epoch 1/100
2/2 [==============================] - 4s 13ms/step - loss: 5512.7637 - accuracy: 0.3600
Epoch 2/100
2/2 [==============================] - 0s 8ms/step - loss: 4507.4561 - accuracy: 0.5200
Epoch 3/100
2/2 [==============================] - 0s 8ms/step - loss: 2504.8584 - accuracy: 0.6600
Epoch 4/100
2/2 [==============================] - 0s 7ms/step - loss: 927.3883 - accuracy: 0.8600
Epoch 5/100
2/2 [==============================] - 0s 8ms/step - loss: 2876.1987 - accuracy: 0.5600
Epoch 6/100
2/2 [==============================] - 0s 7ms/step - loss: 2329.9368 - accuracy: 0.7000
Epoch 7/100
2/2 [==============================] - 0s 9ms/step - loss: 1402.5121 - accuracy: 0.7600
Epoch 8/100
2/2 [==============================] - 0s 9ms/step - loss: 2759.8269 - accuracy: 0.7400
Epoch 9/100
2/2 [==============================] - 0s 8ms/step - loss: 1869.2312 - accuracy: 0.8200
Epoch 10/100
2/2 [==============================] - 0s 8ms/step - loss: 1393.1204 - accuracy: 0.8000

In [16]:

test_dataset = test_dataset.batch(50)
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

200/200 - 0s - loss: 389.8174 - accuracy: 0.8266 - 498ms/epoch - 2ms/step
Test Loss: 389.81744384765625, Test Accuracy: 0.8265612721443176


In [17]:

predictions = model.predict(test_dataset)
for pred, real in zip(predictions, labels):
    predicted_label = (pred > 0.5).astype(int)
    print(f"Predicted: {predicted_label};    Real: {real}")
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

200/200 [==============================] - 1s 2ms/step
Predicted: [0];    Real: 0
Predicted: [0];    Real: 1
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 1
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 1
Predicted: [0];    Real: 0
Predicted: [0];    Real: 1
Predicted: [0];    Real: 1
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [1];    Real: 1
Predicted: [0];    Real: 0
Predicted: [0];    Real: 1
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
Predicted: [0];    Real: 0
